# 전력 소비량 예측 - 핵심 EDA

**목표**: 최종 모델에 사용될 핵심 피처들의 효과성 검증

**분석 내용**:
1. 온도와 전력소비량의 비선형 관계
2. 휴일 효과 분석
3. 시간/요일 패턴 분석
4. 온도 기반 지표(CDH, CDD, THI) 효과
5. 건물별 특성 차이

## 1. 환경 설정

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')
sns.set_palette('husl')

print("✅ 환경 설정 완료")

## 2. 데이터 로드 및 기본 전처리

In [ ]:
# 데이터 로드
DATA_DIR = './data'
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), encoding='utf-8-sig')
building_info = pd.read_csv(os.path.join(DATA_DIR, 'building_info.csv'), encoding='utf-8-sig')

# 날짜 변환
train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')

# 시간 파생 변수
train['hour'] = train['일시'].dt.hour
train['day_of_week'] = train['일시'].dt.dayofweek
train['day'] = train['일시'].dt.day
train['month'] = train['일시'].dt.month
train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)
train['date'] = train['일시'].dt.date

# 건물 정보 병합
train = train.merge(building_info, on='건물번호', how='left')

print(f"✅ Train shape: {train.shape}")
print(f"   데이터 기간: {train['일시'].min()} ~ {train['일시'].max()}")

## 3. 온도와 전력소비량의 비선형 관계

**가설**: 온도가 높아질수록 냉방 전력 수요가 급격히 증가 → 제곱항 필요

In [ ]:
# 온도 구간별 평균 전력소비량
temp_bins = np.arange(15, 40, 1)
train['temp_bin'] = pd.cut(train['기온(°C)'], bins=temp_bins)
temp_power = train.groupby('temp_bin')['전력소비량(kWh)'].agg(['mean', 'std', 'count']).reset_index()
temp_power['temp_center'] = temp_power['temp_bin'].apply(lambda x: x.mid if pd.notna(x) else np.nan)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 선형 관계
axes[0].scatter(temp_power['temp_center'], temp_power['mean'], s=100, alpha=0.7, edgecolor='black')
axes[0].set_title('기온 vs 평균 전력소비량 (선형 척도)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('기온 (°C)', fontsize=12)
axes[0].set_ylabel('평균 전력소비량 (kWh)', fontsize=12)
axes[0].grid(alpha=0.3)

# 제곱 관계
axes[1].scatter(temp_power['temp_center']**2, temp_power['mean'], s=100, alpha=0.7, 
               edgecolor='black', color='orange')
axes[1].set_title('기온² vs 평균 전력소비량 (더 선형적)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('기온² (°C²)', fontsize=12)
axes[1].set_ylabel('평균 전력소비량 (kWh)', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 상관계수 비교
corr_temp = train[['기온(°C)', '전력소비량(kWh)']].corr().iloc[0, 1]
train['기온²'] = train['기온(°C)'] ** 2
corr_temp_sq = train[['기온²', '전력소비량(kWh)']].corr().iloc[0, 1]

print(f"기온 vs 전력소비량 상관계수: {corr_temp:.4f}")
print(f"기온² vs 전력소비량 상관계수: {corr_temp_sq:.4f}")
print(f"\n📊 결론: 제곱항(temperature_squared)을 추가하면 비선형 관계를 더 잘 포착할 수 있음")

## 4. 휴일 효과 분석

**가설**: 건물 타입별로 휴일/평일 전력 소비 패턴이 크게 다름

In [ ]:
# 공휴일 설정 (2024년)
kr_holidays = pd.to_datetime(['2024-06-06', '2024-08-15'])
train['is_holiday'] = train['date'].isin(kr_holidays.date) | train['is_weekend'].astype(bool)

# 건물 타입별 평일/휴일 전력소비량 비교
holiday_analysis = train.groupby(['건물유형', 'is_holiday'])['전력소비량(kWh)'].mean().unstack()
holiday_analysis.columns = ['평일', '휴일']
holiday_analysis['차이율(%)'] = ((holiday_analysis['휴일'] - holiday_analysis['평일']) / holiday_analysis['평일'] * 100).round(2)

print("건물 타입별 평일/휴일 평균 전력소비량 비교:")
print(holiday_analysis.sort_values('차이율(%)', ascending=False))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 막대 그래프
holiday_analysis[['평일', '휴일']].plot(kind='bar', ax=axes[0], rot=45, width=0.8)
axes[0].set_title('건물 타입별 평일/휴일 평균 전력소비량', fontsize=14, fontweight='bold')
axes[0].set_xlabel('건물 유형', fontsize=12)
axes[0].set_ylabel('평균 전력소비량 (kWh)', fontsize=12)
axes[0].legend(['평일', '휴일'], fontsize=11)
axes[0].grid(alpha=0.3, axis='y')

# 차이율
holiday_analysis['차이율(%)'].plot(kind='bar', ax=axes[1], color='coral', rot=45, width=0.8)
axes[1].set_title('건물 타입별 평일 대비 휴일 전력소비량 차이', fontsize=14, fontweight='bold')
axes[1].set_xlabel('건물 유형', fontsize=12)
axes[1].set_ylabel('차이율 (%)', fontsize=12)
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n📊 결론: 건물 타입에 따라 휴일 효과가 크게 다르므로 holiday 피처가 필수적")

## 5. 시간대별/요일별 패턴 분석

**가설**: 시간대와 요일에 따라 전력 소비 패턴이 명확하게 다름 → 타겟 통계 피처 필요

In [ ]:
# 샘플 건물 3개 선택 (서로 다른 타입)
sample_buildings = train.groupby('건물유형')['건물번호'].first().values[:3]

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for idx, building in enumerate(sample_buildings):
    building_data = train[train['건물번호'] == building]
    building_type = building_data['건물유형'].iloc[0]
    
    # 요일-시간대별 평균 전력소비량
    pivot_data = building_data.pivot_table(
        values='전력소비량(kWh)', 
        index='hour', 
        columns='day_of_week', 
        aggfunc='mean'
    )
    
    sns.heatmap(pivot_data.T, annot=False, cmap='YlOrRd', ax=axes[idx], 
               cbar_kws={'label': '평균 전력소비량 (kWh)'})
    axes[idx].set_title(f'건물 {building} ({building_type}) - 요일×시간대 전력소비 패턴', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('시간', fontsize=11)
    axes[idx].set_ylabel('요일 (0=월요일)', fontsize=11)

plt.tight_layout()
plt.show()

# 패턴의 변동성 계산
dow_hour_std = train.groupby(['건물번호', 'hour', 'day_of_week'])['전력소비량(kWh)'].std()
overall_std = train.groupby('건물번호')['전력소비량(kWh)'].std()

print(f"\n📊 결론: 시간대와 요일에 따라 뚜렷한 패턴이 존재")
print(f"   → dow_hour_mean/std, hour_mean/std 등 타겟 통계 피처가 매우 유용")

## 6. 온도 기반 지표 분석

**가설**: CDH(냉방도시), THI(온열지수) 등이 단순 온도보다 전력소비량을 더 잘 설명

In [ ]:
# CDH (Cooling Degree Hours) 계산 - 간단 버전
def calc_cdh_simple(temp, base_temp=26, window=12):
    cdh_list = []
    for i in range(len(temp)):
        if i < window:
            cdh = np.sum(np.maximum(temp[:i+1] - base_temp, 0))
        else:
            cdh = np.sum(np.maximum(temp[i-window+1:i+1] - base_temp, 0))
        cdh_list.append(cdh)
    return cdh_list

# 샘플 건물에 대해 CDH 계산
sample_building = train['건물번호'].value_counts().index[0]
sample_data = train[train['건물번호'] == sample_building].sort_values('일시').head(500)
sample_data['CDH'] = calc_cdh_simple(sample_data['기온(°C)'].values)

# THI (Temperature Humidity Index) 계산
sample_data['THI'] = (9/5 * sample_data['기온(°C)'] 
                      - 0.55 * (1 - sample_data['습도(%)'] / 100) 
                      * (9/5 * sample_data['기온(°C)'] - 26) + 32)

# 상관계수 비교
corr_comparison = pd.DataFrame({
    '피처': ['기온', '습도', 'CDH', 'THI'],
    '상관계수': [
        sample_data[['기온(°C)', '전력소비량(kWh)']].corr().iloc[0, 1],
        sample_data[['습도(%)', '전력소비량(kWh)']].corr().iloc[0, 1],
        sample_data[['CDH', '전력소비량(kWh)']].corr().iloc[0, 1],
        sample_data[['THI', '전력소비량(kWh)']].corr().iloc[0, 1]
    ]
})
corr_comparison['절대값'] = corr_comparison['상관계수'].abs()
corr_comparison = corr_comparison.sort_values('절대값', ascending=False)

print(f"건물 {sample_building} - 피처별 전력소비량 상관계수:")
print(corr_comparison)

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].scatter(sample_data['기온(°C)'], sample_data['전력소비량(kWh)'], alpha=0.5)
axes[0, 0].set_title(f'기온 vs 전력소비량 (상관: {corr_comparison.iloc[3, 1]:.3f})', fontsize=12)
axes[0, 0].set_xlabel('기온 (°C)')
axes[0, 0].set_ylabel('전력소비량 (kWh)')

axes[0, 1].scatter(sample_data['습도(%)'], sample_data['전력소비량(kWh)'], alpha=0.5, color='green')
axes[0, 1].set_title(f'습도 vs 전력소비량 (상관: {corr_comparison.iloc[2, 1]:.3f})', fontsize=12)
axes[0, 1].set_xlabel('습도 (%)')
axes[0, 1].set_ylabel('전력소비량 (kWh)')

axes[1, 0].scatter(sample_data['CDH'], sample_data['전력소비량(kWh)'], alpha=0.5, color='orange')
axes[1, 0].set_title(f'CDH vs 전력소비량 (상관: {corr_comparison.iloc[1, 1]:.3f})', fontsize=12)
axes[1, 0].set_xlabel('CDH (Cooling Degree Hours)')
axes[1, 0].set_ylabel('전력소비량 (kWh)')

axes[1, 1].scatter(sample_data['THI'], sample_data['전력소비량(kWh)'], alpha=0.5, color='red')
axes[1, 1].set_title(f'THI vs 전력소비량 (상관: {corr_comparison.iloc[0, 1]:.3f})', fontsize=12)
axes[1, 1].set_xlabel('THI (Temperature Humidity Index)')
axes[1, 1].set_ylabel('전력소비량 (kWh)')

plt.tight_layout()
plt.show()

print(f"\n📊 결론: CDH, THI 등 복합 지표가 단순 온도/습도보다 전력소비량을 더 잘 설명")

## 7. 순환 패턴 (Cyclic Features)

**가설**: 시간과 날짜는 순환적이므로 sin/cos 변환이 필요

In [ ]:
# 순환 피처 생성
train['sin_hour'] = np.sin(2 * np.pi * train['hour'] / 24)
train['cos_hour'] = np.cos(2 * np.pi * train['hour'] / 24)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 원형 시간
axes[0].scatter(train['cos_hour'], train['sin_hour'], c=train['hour'], 
               cmap='hsv', s=1, alpha=0.5)
axes[0].set_title('시간의 순환적 표현 (sin, cos)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('cos(hour)', fontsize=12)
axes[0].set_ylabel('sin(hour)', fontsize=12)
axes[0].set_aspect('equal')
axes[0].grid(alpha=0.3)

# 시간별 평균 전력소비량
hourly_avg = train.groupby('hour')['전력소비량(kWh)'].mean()
axes[1].plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2)
axes[1].set_title('시간대별 평균 전력소비량 (순환 패턴)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('시간', fontsize=12)
axes[1].set_ylabel('평균 전력소비량 (kWh)', fontsize=12)
axes[1].set_xticks(range(0, 24, 3))
axes[1].grid(alpha=0.3)

# 선형 hour vs 순환 hour 비교
corr_linear = train[['hour', '전력소비량(kWh)']].corr().iloc[0, 1]
corr_sin = train[['sin_hour', '전력소비량(kWh)']].corr().iloc[0, 1]
corr_cos = train[['cos_hour', '전력소비량(kWh)']].corr().iloc[0, 1]

comparison = pd.DataFrame({
    '피처': ['hour (선형)', 'sin_hour', 'cos_hour'],
    '상관계수': [corr_linear, corr_sin, corr_cos]
})
comparison['절대값'] = comparison['상관계수'].abs()
comparison.plot(x='피처', y='절대값', kind='bar', ax=axes[2], legend=False)
axes[2].set_title('시간 표현 방식별 전력소비량 상관관계', fontsize=14, fontweight='bold')
axes[2].set_ylabel('상관계수 절대값', fontsize=12)
axes[2].set_xticklabels(comparison['피처'], rotation=45, ha='right')
axes[2].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n피처별 상관계수:")
print(comparison)
print(f"\n📊 결론: 시간은 순환적 특성을 가지므로 sin/cos 변환(cyclic features)이 유용")

## 8. 건물별 특성 차이

**가설**: 건물마다 전력 소비 패턴이 크게 다르므로 개별 모델링이 효과적

In [ ]:
# 건물별 통계
building_stats = train.groupby('건물번호')['전력소비량(kWh)'].agg([
    'mean', 'std', 'min', 'max'
]).reset_index()
building_stats['변동계수(CV)'] = building_stats['std'] / building_stats['mean']
building_stats = building_stats.sort_values('mean', ascending=False)

print("건물별 전력소비량 통계 (상위 10개):")
print(building_stats.head(10))

# 시각화
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# 건물별 평균 전력소비량
axes[0].bar(range(len(building_stats)), building_stats['mean'], alpha=0.7, edgecolor='black')
axes[0].set_title('건물별 평균 전력소비량', fontsize=14, fontweight='bold')
axes[0].set_xlabel('건물 (평균 소비량 내림차순)', fontsize=12)
axes[0].set_ylabel('평균 전력소비량 (kWh)', fontsize=12)
axes[0].grid(alpha=0.3, axis='y')

# 건물별 변동계수
axes[1].bar(range(len(building_stats)), building_stats['변동계수(CV)'], 
           alpha=0.7, edgecolor='black', color='orange')
axes[1].set_title('건물별 변동계수 (표준편차/평균)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('건물', fontsize=12)
axes[1].set_ylabel('변동계수', fontsize=12)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n평균 소비량 범위: {building_stats['mean'].min():.2f} ~ {building_stats['mean'].max():.2f} kWh")
print(f"평균 소비량 표준편차: {building_stats['mean'].std():.2f} kWh")
print(f"\n📊 결론: 건물 간 차이가 매우 크므로 건물별 개별 모델링이 효과적")

## 9. EDA 종합 결론

### 핵심 발견사항 및 피처 설계 근거

1. **온도의 비선형 효과**
   - 온도가 높아질수록 전력소비량이 급격히 증가
   - → `temperature_squared`, `humidity_squared` 피처 추가 필요

2. **휴일 효과**
   - 건물 타입별로 휴일 전력소비 패턴이 크게 상이
   - → `holiday` 피처 필수, 건물별 세밀한 휴일 설정 필요

3. **시간/요일 패턴**
   - 시간대와 요일에 따라 명확한 전력소비 패턴 존재
   - → `dow_hour_mean/std`, `hour_mean/std`, `month_hour_mean/std` 등 타겟 통계 피처 매우 유용

4. **온도 기반 복합 지표**
   - CDH, THI 등이 단순 온도보다 전력소비량을 더 잘 설명
   - → `CDH`, `CDD`, `THI`, `WCT` 피처 추가

5. **순환적 특성**
   - 시간과 날짜는 순환적 패턴을 가짐
   - → `sin_hour`, `cos_hour`, `sin_doy`, `cos_doy`, `summer_sin/cos` 필요

6. **건물별 이질성**
   - 건물마다 전력소비 패턴이 크게 다름
   - → 건물별 개별 모델링 전략 필요

### 제외 피처
- `sunshine`, `solar_radiation`: 상관관계 낮음
- `solar_power_capacity`, `ess_capacity`, `pcs_capacity`: 대부분 0이거나 결측값

### 다음 단계
- Feature Selection으로 최종 피처 선정
- 이상치 분석 및 처리
- 모델 학습 및 검증